<a href="https://colab.research.google.com/github/narendra-mds/CS5660/blob/main/CS5660_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np
import tarfile
import os

In [24]:
current_directory = os.getcwd()

print("Current working directory:", current_directory)

Current working directory: /content


In [53]:
def load_fvecs_from_tar(tar_filename, fvecs_filename):
    with tarfile.open(tar_filename, 'r') as tar:
        # Extract the fvecs file from the tar archive
        fvecs_file = tar.extractfile(fvecs_filename)
        with fvecs_file as f:
          fv = np.frombuffer(f.read(), dtype=np.float32)
          if fv.size == 0:
            return np.zeros((0, 0))
        dim = fv.view(np.int32)[0]
        fv = fv.reshape(-1, 1 + dim)
        if not all(fv.view(np.int32)[:, 0] == dim):
          raise IOError("Non-uniform vector sizes in " + fvecs_file)
        fv = fv[:, 1:]
        fv = fv.copy()
    return fv

In [58]:
def load_ivecs_from_tar(tar_filename, ivecs_filename):
    with tarfile.open(tar_filename, 'r') as tar:
        # Extract the ivecs file from the tar archive
        ivecs_file = tar.extractfile(ivecs_filename)
        with ivecs_file as f:
          a = np.frombuffer(f.read(), dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().reshape(-1)

In [54]:
tar_file = 'groupFungus_k64_nclass10_nex10.tgz'
folder = 'example_data'
fvecs_filename_images = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Xtrain.fvecs')
ivecs_filename_labels = os.path.join(folder, 'groupFungus_k64_nclass10_nex10_Ltrain.ivecs')

In [55]:
image_features = load_fvecs_from_tar(tar_file, fvecs_filename_images)

In [57]:
image_features.shape

(100, 4096)

In [59]:
image_labels = load_ivecs_from_tar(tar_file, ivecs_filename_labels)

In [60]:
image_labels.shape

(100,)